## 라이브러리 불러오기

In [1]:
import pandas as pd
# DataFrame 화면 출력                                                # DataFrame 화면 출력 설정이 변경되어 있습니다 #
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None # default = 60

import datetime

# API 호출
# ## https://data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15057440
from urllib.request import Request, urlopen
# Request https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.Request
# urlopen https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.urlopen
from urllib.parse import urlencode, quote_plus
# urlenocde https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.urlencode
# quote_plus https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.quote_plus
from urllib.parse import unquote
import requests

# XML 파싱
import xml.etree.ElementTree as ET

## 기본값 설정하기

In [2]:
routeId = 204000046
year = 2020
month = 11
day = 16

## Key Unquote

In [3]:
KEY = 'yEaR%2F3MDedRSlVJL%2F2pxnVg0yre1N5VF3RZ%2FUAt56MJ7J2mNpfqhUvy05pXV0uhHTVY7DbyCR8xmMaDdYga67Q%3D%3D' # 종현
# apiKEY = requests.utils.unquote(KEY)
apiKey = unquote(KEY)
# print(apiKey)
KEYSW = 'M%2B4%2FqUiadT8X8PhgFjaQLDu%2BIOgPMURfGsOX%2FmVxwHQVJgnVR%2FMPjDYXkuQNwUFbZXlfnX5Lls3SUCiCLIFjgQ%3D%3D'
apiKeySW = unquote(KEYSW)

## API 호출

In [4]:
# 노선정보항목조회 busrouteservice/info

# 해당 노선에 대한 노선번호, 기점/종점 정류소, 첫차/막차시간, 배차간격, 운행업체 등의 운행계획 정보를 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/station'
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : routeId })
request = Request(url + queryParams)
request.get_method = lambda: 'GET'

# API를 호출하여 XML 형식으로 된 string 데이터를 변수 "oneLineXML" 에 저장합니다.
oneLineXML = urlopen(request).read().decode('utf8')
# oneLineXML

## XML 파싱

In [5]:
# xtree는 "openapi" 도메인 API 호출에서 "headerCd"에 상관없이 3개의 태그를 갖습니다.
# [comMsgHeader, msgHeader, msgBody]
xtree = ET.fromstring(oneLineXML)
# for branch in xtree:
#     print(branch)
msgBody = xtree[2]

tagList = ["stationSeq", "stationId", "stationName"]
stationList = []
for branch in msgBody:
    stationList.append([branch.find(tag).text for tag in tagList])

# station_df의 index가 0부터 시작하는 문제를 해결 하기 위해 rangeIndex를 지정하였습니다.
stationCount = len(stationList) # 뒤에서 다시 사용되므로 함수로 선언시 주의
rangeIndex = pd.RangeIndex(start=1, stop=stationCount+1)

station_df = pd.DataFrame(stationList, columns=tagList, index=rangeIndex)
station_df = station_df.astype({
    "stationSeq" : "int32",
    "stationId" : "int32", 
    "stationName" : "string"
    
})
station_df

,stationSeq,stationId,stationName
1,1,206000010,운중동먹거리촌
2,2,206000009,운중동푸르지오하임
3,3,206000658,한빛교회.월든힐스아파트
4,4,206000558,운중초등학교
5,5,206000007,운중중학교
6,6,206000006,운중동행정복지센터
7,7,206000005,뫼루니육교
8,8,206000004,판교원마을1.2단지.판교도서관
9,9,206000555,판교청소년수련관.판교종합사회복지관
10,10,206000003,한림아파트


## 데이터 불러오기

In [6]:
# routeId 별 정류소 도착시간을 나타내는 2차원 배열을 생성 할 수 있습니다
# 마지막 정류소 번호가 필요합니다 : 경유정류소목록조회 API 서비스를 이용하여 가져옵니다.
# 중간 또는 처음과 끝에 발생하는 결측치를 채워야 합니다.


rootPath = 'C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/buslocationservice/'
outputPath = rootPath + str(routeId) + '/' + str(routeId) + '_' + str(year)[-2:] + '-' + str(month) + '-' + str(day) + '.txt'

itemTagList = ['endBus', 'lowPlate', 'plateNo', 'plateType', 'remainSeatCnt', 'routeId', 'stationId', 'stationSeq', 'Nan']
# itemTagList = ['endBus', 'plateNo', 'plateType', 'remainSeatCnt', 'routeId', 'stationId', 'stationSeq', 'Nan']
itemTagList.insert(0, 'DateTime')
_df = pd.read_csv(outputPath, sep=' ', skiprows=[0], names=itemTagList)
_df = _df.drop(["Nan"], axis=1)
_df = _df.sort_values(['plateNo'])
_df = _df.dropna()
# _df = _df.drop(_df[_df["endBus"]=="API"].index)

# string을 datetime객체로 변환하기
_df['DateTime'] = pd.to_datetime(_df['DateTime'])
# dateTimeObj = datetime.datetime.strptime(_df['DateTime'], '%Y-%m-%dT%H:%M:%S.%f%z')
# _df['DateTime'] = _df.DateTime.str.split('.').str[0]
# _df['DateTime'] = _df.DateTime.str.replace('T', ' ')

_df = _df.astype({
    'endBus': 'int32', 
    'lowPlate': 'int32', 
    'plateNo': 'string', 
    'plateType': 'int32',
    'remainSeatCnt': 'int32', 
    'routeId': 'int32', 
    'stationId': 'int32', 
    'stationSeq': 'int32'
})
_df.dtypes

DateTime         datetime64[ns, pytz.FixedOffset(540)]
endBus                                           int32
lowPlate                                         int32
plateNo                                         string
plateType                                        int32
remainSeatCnt                                    int32
routeId                                          int32
stationId                                        int32
stationSeq                                       int32
dtype: object

In [7]:
# 차량 운행기록을 "plateNo"를 기준으로 구분합니다.
plateNoList = _df.groupby(['plateNo']).size().index

In [8]:
# 정류장의 수 만큼 행을 갖는 빈 데이터프레임을 생성합니다. 변수 rangeIndex는 [XML파싱]에서 선언되어 있습니다.
oneDay_df = pd.DataFrame(index = rangeIndex)

# 각 차량 번호를 기준으로 반복문을 수행합니다.
for plateNo in plateNoList:
    
    # 특정 차량번호와 같은 운행기록을 "DateTime을 기준으로 오름차순으로 정렬합니다."
    sameBusList = _df[_df['plateNo'] == plateNo].sort_values(['DateTime']).reset_index(drop=True)
#     print(sameBusList)
    
    
    # 버스는 뒤로가지 않으므로
    # "stationSeq"가 작아지는 인덱스 i의 리스트를 만듭니다.
    indexList = []
    for i in range(1,len(sameBusList)):
        if(sameBusList.iloc[i-1]['stationSeq'] > sameBusList.iloc[i]['stationSeq']):
            indexList.append(i)
#             print((sameBusList.iloc[i-1]['stationSeq'] , sameBusList.iloc[i]['stationSeq']))
            
    indexList.append(len(sameBusList))
#     print(indexList)
    
    # 첫번재 인덱스, 인덱스의 리스트, 마지막 인덱스를 조합하여 호차별로 운행기록을 구분합니다.
    index = 0
    tupleList = []
    for i in indexList:
        tupleList.append((index, i))
        small_df = sameBusList.iloc[index:i].groupby(by=['stationSeq']).min()
        
        # 정류소 데이터프레임에 합칩니다.
        merge_df = pd.merge(station_df, small_df, how='outer', on='stationId')
        merge_df = merge_df.set_index(keys='stationSeq')
        
        
        if(1):
            # 결측치를 처리합니다.
            # df.interpolate(method='linear' or 'polynomial')
            # 시간에 대한 결측치 처리
            merge_df['DateTime'] = merge_df['DateTime'].interpolate(method='backfill')
            merge_df['DateTime'] = merge_df['DateTime'].interpolate(method='pad')

            # 빈좌석에 대한 결측치 처리, limit 값이 다름에 주의
            merge_df['remainSeatCnt'] = merge_df['remainSeatCnt'].interpolate(method='linear', limit=2, limit_direction='both')

            merge_df['endBus'] = merge_df['endBus'].interpolate(method='pad', limit=2)        
            merge_df['endBus'] = merge_df['endBus'].interpolate(method='backfill', limit=2)

            merge_df['lowPlate'] = merge_df['lowPlate'].interpolate(method='pad', limit=2)        
            merge_df['lowPlate'] = merge_df['lowPlate'].interpolate(method='backfill', limit=2)

            merge_df['plateType'] = merge_df['plateType'].interpolate(method='pad', limit=2)        
            merge_df['plateType'] = merge_df['plateType'].interpolate(method='backfill', limit=2)

            merge_df['routeId'] = merge_df['routeId'].interpolate(method='pad', limit=2)        
            merge_df['routeId'] = merge_df['routeId'].interpolate(method='backfill', limit=2)

        oneDay_df[plateNo+'_'+str(index)] = merge_df['DateTime']
        

#         display(merge_df)
        index=i
#         break;
#     break;

oneDay_df = oneDay_df.sort_values(by=1, axis=1)
# oneDay_df

In [9]:
# station_df 와 oneDay_df를 합칩니다. datetime이 아닌 다른 타입의 오브젝트가 행에 붙게 되므로 지금부터는 편집이 어렵습니다.
oneDay_df = pd.concat([station_df,oneDay_df], axis=1)
finalOneDay_df = oneDay_df.set_index('stationSeq', drop=True)

In [10]:
finalOneDay_df

,stationId,stationName,경기78아1147_0,경기70아8789_0,경기70아8601_0,경기70아7474_0,경기77바1198_0,경기70아8880_0,경기78아1117_0,경기70아8629_0,경기78아1178_0,경기77바1097_0,경기70아7482_0,경기70아7361_0,경기70아7474_6,경기77바1198_30,경기70아8601_54,경기78아1147_70,경기78아1117_99,경기70아8789_119,경기70아8880_129,경기70아8629_127,경기78아1178_121,경기77바1097_123,경기70아7482_119,경기70아7361_109,경기70아7474_125,경기77바1198_152,경기70아8601_169,경기78아1147_179,경기78아1117_213,경기70아8789_243
stationSeq,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,206000010,운중동먹거리촌,2020-11-16 17:02:41.571481+09:00,2020-11-16 17:02:41.571551+09:00,2020-11-16 17:02:41.571571+09:00,2020-11-16 17:02:41.571587+09:00,2020-11-16 17:02:41.571603+09:00,2020-11-16 17:02:41.571618+09:00,2020-11-16 17:02:41.571632+09:00,2020-11-16 17:12:21.120892+09:00,2020-11-16 17:25:52.488106+09:00,2020-11-16 17:37:27.554241+09:00,2020-11-16 17:49:01.852074+09:00,2020-11-16 18:00:38.238873+09:00,2020-11-16 18:17:08.172974+09:00,2020-11-16 18:31:36.565070+09:00,2020-11-16 18:47:03.712993+09:00,2020-11-16 19:00:33.929671+09:00,2020-11-16 19:13:09.900622+09:00,2020-11-16 19:27:41.017280+09:00,2020-11-16 19:36:21.556856+09:00,2020-11-16 19:51:48.511036+09:00,2020-11-16 20:06:17.318082+09:00,2020-11-16 20:21:44.970177+09:00,2020-11-16 20:41:04.356698+09:00,2020-11-16 21:01:23.551333+09:00,2020-11-16 21:21:39.840063+09:00,2020-11-16 21:40:36.579705+09:00,2020-11-16 22:01:53.649237+09:00,2020-11-16 22:27:01.866904+09:00,2020-11-16 22:51:11.472466+09:00,2020-11-16 23:11:34.206716+09:00
2,206000009,운중동푸르지오하임,2020-11-16 17:02:41.571481+09:00,2020-11-16 17:02:41.571551+09:00,2020-11-16 17:02:41.571571+09:00,2020-11-16 17:02:41.571587+09:00,2020-11-16 17:02:41.571603+09:00,2020-11-16 17:02:41.571618+09:00,2020-11-16 17:02:41.571632+09:00,2020-11-16 17:12:21.120892+09:00,2020-11-16 17:25:52.488106+09:00,2020-11-16 17:37:27.554241+09:00,2020-11-16 17:49:01.852074+09:00,2020-11-16 18:00:38.238873+09:00,2020-11-16 18:17:08.172974+09:00,2020-11-16 18:31:36.565070+09:00,2020-11-16 18:47:03.712993+09:00,2020-11-16 19:00:33.929671+09:00,2020-11-16 19:13:09.900622+09:00,2020-11-16 19:27:41.017280+09:00,2020-11-16 19:36:21.556856+09:00,2020-11-16 19:51:48.511036+09:00,2020-11-16 20:06:17.318082+09:00,2020-11-16 20:21:44.970177+09:00,2020-11-16 20:41:04.356698+09:00,2020-11-16 21:01:23.551333+09:00,2020-11-16 21:21:39.840063+09:00,2020-11-16 21:40:36.579705+09:00,2020-11-16 22:01:53.649237+09:00,2020-11-16 22:27:01.866904+09:00,2020-11-16 22:51:11.472466+09:00,2020-11-16 23:11:34.206716+09:00
3,206000658,한빛교회.월든힐스아파트,2020-11-16 17:02:41.571481+09:00,2020-11-16 17:02:41.571551+09:00,2020-11-16 17:02:41.571571+09:00,2020-11-16 17:02:41.571587+09:00,2020-11-16 17:02:41.571603+09:00,2020-11-16 17:02:41.571618+09:00,2020-11-16 17:02:41.571632+09:00,2020-11-16 17:13:19.014879+09:00,2020-11-16 17:26:50.342361+09:00,2020-11-16 17:37:27.554241+09:00,2020-11-16 17:49:01.852074+09:00,2020-11-16 18:01:36.342306+09:00,2020-11-16 18:17:08.172974+09:00,2020-11-16 18:31:36.565070+09:00,2020-11-16 18:47:03.712993+09:00,2020-11-16 19:01:32.043544+09:00,2020-11-16 19:14:07.814584+09:00,2020-11-16 19:27:41.017280+09:00,2020-11-16 19:37:19.510788+09:00,2020-11-16 19:51:48.511036+09:00,2020-11-16 20:07:15.321929+09:00,2020-11-16 20:21:44.970177+09:00,2020-11-16 20:42:02.180735+09:00,2020-11-16 21:01:23.551333+09:00,2020-11-16 21:22:37.686087+09:00,2020-11-16 21:41:34.573683+09:00,2020-11-16 22:02:51.723673+09:00,2020-11-16 22:27:01.866904+09:00,2020-11-16 22:52:09.336367+09:00,2020-11-16 23:12:32.240816+09:00
4,206000558,운중초등학교,2020-11-16 17:02:41.571481+09:00,2020-11-16 17:02:41.571551+09:00,2020-11-16 17:02:41.571571+09:00,2020-11-16 17:02:41.571587+09:00,2020-11-16 17:02:41.571603+09:00,2020-11-16 17:02:41.571618+09:00,2020-11-16 17:02:41.571632+09:00,2020-11-16 17:15:15.183102+09:00,2020-11-16 17:27:48.396557+09:00,2020-11-16 17:38:25.308177+09:00,2020-11-16 17:50:57.340189+09:00,2020-11-16 18:03:32.099485+09:00,2020-11-16 18:18:05.987019+09:00,2020-11-16 18:33:32.483703+09:00,2020-11-16 18:48:01.487190